## Strategy debugging example

Debugging a strategy can be time-consuming. FreqTrade offers helper functions to visualize raw data.

## Setup

### Change directory
Jupyter notebooks execute from the current directory. Change working directory to the project root so that relative paths work.

In [ ]:
import os
from pathlib import Path

# Modify this cell to insure that the output shows the correct path.
project_root = "somedir/freqtrade"
i=0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

### Fix asycronous execution
Jupyter notebooks get confused with async operations. This fixes the problem.

In [ ]:
import nest_asyncio

# Fix asyncio for Jupyter
nest_asyncio.apply()

### Generate configuration
This combines all config files into a single dictionary. Individual options can be manipulated with dict indexing.
Watch out for type conversion pitfalls. The config dict created here has string values. More work needs to be done to automatically coerce strings to the expected object types. You will need to perform type conversion when passing function arguments as demonstrated in the following sections.

In [ ]:
from freqtrade.configuration import Configuration
from freqtrade.configuration.timerange import TimeRange

# Load configuration
# Specify values for use in this script
############### Customize to match your needs. ##################
config_files = [
    Path('user_data', 'user_repo', 'config.json'),
    Path(Path.home(), '.freqtrade', 'exchange-config.json')
]
# Create config object
config = Configuration.from_files(config_files)

############### Customize to match your needs. ##################
# Define some constants
ticker_interval = "5m"
# Path to user data
user_data_dir = Path('user_data')
# Location of the ticker data
datadir = Path(user_data_dir, 'data/binance')
# Name of the strategy class
strategy_name = 'NewStrategy'
# Location of the strategy
strategy_path = Path(user_data_dir, 'user_repo/strategies')
# Specify backtest results to load
trade_source = 'file'
exportfilename = Path(user_data_dir, 'backtest_results/backtest-result.json')
db_url = 'sqlite://'
# Specify timerange to test
timerange = '-1000'
# Pair to analyze - Only use one pair here
pair = "ETH/BTC"

### Generate config argument for CLI
CLI commands can be executed from a notebook cell by including `!` before the command. This combines all config files into a variable that can be called in a cli command as `{conf}`

In [ ]:
from itertools import chain

# Create config string for use in cli commands
conf = " ".join(
    list(chain.from_iterable([['-c', str(file)] for file in config_files])))

# Example cli command
!freqtrade {conf} backtesting --help

### Configure logging
This imports the freqtrade logger format and displays logging messages from the internal functions.

In [ ]:
import logging

from freqtrade.loggers import setup_logging

# Configure logging
logger = logging.getLogger()
setup_logging(config)
logger.setLevel(logging.INFO)
logger.info(f'conf: {conf}')

# Show config in memory
logger.info(json.dumps(config, indent=1))

## Visualize buy/sell signals
This uses the values set in the configuration section to:
* Load the history of the specified pair
* Load the specified strategy
* Generate buy and sell signals produced by the specified strategy
* Plot the results


*Note: `data.head()` may include empty values for indicators that require a startup period. This is expected behavior. For example `ma5` requires 5 candles before computing the first average*

### Possible problems

* Columns with NaN values at the end of the dataframe
* Columns used in `crossed*()` functions with completely different units

### Comparison with full backtest

* having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
* Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  

In [ ]:
from pathlib import Path

import pandas as pd

from freqtrade.data.btanalysis import load_trades
from freqtrade.data.history import load_pair_history
from freqtrade.resolvers import StrategyResolver
from freqtrade.plot.plotting import extract_trades_of_period, generate_candlestick_graph
# Load ticker history
tickers = load_pair_history(pair=pair,
                            ticker_interval=ticker_interval,
                            datadir=datadir,
                            timerange=TimeRange.parse_timerange(timerange))

# Confirm success
print("Loaded " + str(len(tickers)) +
      f" rows of data for {pair} from {datadir}")

# Load strategy
strategy = StrategyResolver({
    'strategy': strategy_name,
    'user_data_dir': user_data_dir,
    'strategy_path': strategy_path
}).strategy

# Generate buy/sell signals using strategy
data = strategy.analyze_ticker(tickers, {'pair': pair})
logger.info(f'Indicators: {list(data)[6:-2]}')

# Collect trades if a backtest has been completed
try:
    trades = load_trades(source=trade_source,
        db_url=db_url,
        exportfilename=exportfilename
    )
    trades = trades.loc[trades['pair'] == pair]
    trades = extract_trades_of_period(data, trades)
except:
    trades = pd.DataFrame()

# Build and display plot
# Specify the indicators to plot as lists
# indicators1 is a list of indicators to overlay on the price chart
# indicators2 is a list of indicators to plot below the price chart
fig = generate_candlestick_graph(pair=pair,
                                 data=data,
                                 trades=trades,
                                 indicators1=['ema20', 'ema50', 'ema100', 'ha_open', 'ha_close'],
                                 indicators2=['macd', 'macdsignal','ao'])

fig.show()
display(data.tail())

### Run Backtest
Once you are happy with your strategy signals, run a backtest.

In [ ]:
# Run backtest
!freqtrade {conf} backtesting --timerange={timerange} --ticker-interval {ticker_interval} --export=trades --export-filename={exportfilename}

Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.